**Name:** Weng Yuqian

**EID:** 55972965

**Kaggle Team Name:** Cccc

# CS5489 - Assignment 1 - Tweet Sentiment Analysis

## Final submission
In this file, put the code that generates your final Kaggle submission. It will be used to verify that your Kaggle submission is reproducible.

In [1]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import csv
random.seed(100)

In [2]:
def read_text_data(fname):
    txtdata = []
    classes = []
    topics  = []
    with open(fname, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            # get the text
            txtdata.append(row[0])
            # get the class (convert to integer)
            if len(row)>1:
                classes.append(row[1])
                topics.append(row[2])
    
    if (len(classes)>0) and (len(txtdata) != len(classes)):        
        raise Exception("mismatched length!")
    
    return (txtdata, classes, topics)

def write_csv_kaggle_sub(fname, Y):
    # fname = file name
    # Y is a list/array with class entries
    
    # header
    tmp = [['Id', 'Prediction']]
    
    # add ID numbers for each Y
    for (i,y) in enumerate(Y):
        tmp2 = [(i+1), y]
        tmp.append(tmp2)
        
    # write CSV file
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(tmp)

In [3]:
# load the data
# (if using Kaggle notebooks you need to include the directory path: /kaggle/input/cs5489-2020b-assignment-1/)
(traintxt, trainY, traintopic) = read_text_data("sanders_tweets_train.txt")
(testtxt, _, _)                = read_text_data("sanders_tweets_test.txt")

print(len(traintxt))
print(len(testtxt))

2396
1028


In [ ]:
### INSERT YOUR CODE HERE

### Feature extraction
- At first, I use tf-idf to du feature extraction

In [4]:
tfidf_vectorizer = feature_extraction.text.TfidfVectorizer (norm='l1', stop_words='english', max_features=850)
features = tfidf_vectorizer.fit_transform(traintxt)
testtf=tfidf_vectorizer.fit_transform(testtxt)

### Multinormial NB
- I split the whole training set into two parts of 8:2, set 8 of them as new training set.
- Then I use the new training set to seleclt best alpha of multinomial naive bayes classifier.

In [22]:
alphas = logspace(-6,3,100)
accuracies = []
X_train, X_test, y_train, y_test = model_selection.train_test_split(features, trainY, test_size = 0.2, random_state = 0)
for al in alphas:
    mmodel_tf = naive_bayes.MultinomialNB(alpha=al)
    mmodel_tf.fit(X_train, y_train)
    tf_predictY = mmodel_tf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, tf_predictY)
    accuracies.append(accuracy)

n_bestalpha = argmax(accuracies)
bestalpha = alphas[n_bestalpha]
bestaccuracy = accuracies[n_bestalpha]
print("best alpha=", bestalpha, ", best accuracy=", bestaccuracy)
mnb_clf = naive_bayes.MultinomialNB(alpha=bestalpha).fit(X_train, y_train)
predictY_mnb = mnb_clf.predict(testtf)
# write_csv_kaggle_sub("my_submission1_MNB.csv", predictY_mnb2)

best alpha= 0.00023101297000831605 , best accuracy= 0.675


- As we can see from the result, the best accuracy is only 0.675, it seems multinormial naive bayes is not performanced well in this dataset.
- The selected best alpha is too small, which means the data is not smooth.
- The final predict result of the test set is also not performanced well.

### SVM
- Then I tried use pipeline to do the svm classifier with count vectorizer and tf-idf transformer.

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
clf = Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', SVC(C=0.99, kernel = 'linear'))])
X_train, X_test, y_train, y_test = model_selection.train_test_split(traintxt, trainY, test_size = 0.2, random_state = 0)
svm_clf=clf.fit(X_train,y_train)
pred=clf.predict(X_test)
acc = metrics.accuracy_score(pred, y_test)
print("test accuracy:",acc)
svm_clf = clf.fit(traintxt, trainY)
predictY_svm = svm_clf.predict(testtxt)
write_csv_kaggle_sub("my_submission.csv", predictY_svm)

test accuracy: 0.7416666666666667


- From the test accuracy we can see, it performanced better than multinormial naive bayes classifier.
- In fact, this result is the best result I get, and also my best submission on kaggle.

In [32]:
parameters = {'vect__max_df': ( 0.6, 0.7, 0.8, 0.9),'vect__max_features': (None, 500, 800),
              'tfidf__use_idf': (True, False),'clf__C': (logspace(-3,3,10))}
gs = model_selection.GridSearchCV(svm_clf, parameters, n_jobs=-1, verbose=1,cv=5)
gs.fit(X_train,y_train)
best_parameters = dict()
best_parameters = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
pred=gs.predict(X_test)
acc = metrics.accuracy_score(pred, y_test)
print("test accuracy:",acc)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  3.8min finished


	clf__C: 2.154434690031882
	tfidf__use_idf: False
	vect__max_df: 0.6
	vect__max_features: None
test accuracy: 0.7666666666666667


- Then I try to optimize the parameters of the svm classifier by using cross validation.
- It shows the test accuracy is greater than before, and also select some best parameters.

In [34]:
predictY_svm2 = gs.predict(testtxt)
# write_csv_kaggle_sub("my_submission3_svm.csv", predictY_svm2)

- But the final result is not as good as before, and I cannot figure out why.

### Gradient boosting
- I use xgboost, i.e. gradient boosting decision tree, with cross validation.
- It can use all CPU cores to build trees in parallel, and the prediction is precise in usual.

In [35]:
import xgboost as xgb
paramsampler = {    
    "colsample_bytree": stats.uniform(0.7, 0.3),  
    "gamma":            stats.uniform(0, 0.5),    
    "max_depth":        stats.randint(2, 6),      
    "subsample":        stats.uniform(0.6, 0.4),  
    "learning_rate":    stats.uniform(.001,1),    
    "n_estimators":     stats.randint(10, 1000),
}
X_train, X_test, y_train, y_test = model_selection.train_test_split(features, trainY, test_size = 0.2, random_state = 0)
xclf = xgb.XGBClassifier(objective="binary:logistic", random_state=4487)
xgbrcv = model_selection.RandomizedSearchCV(xclf, param_distributions=paramsampler, 
                            random_state=4487, n_iter=200, cv=5, 
                            verbose=1, n_jobs=-1)
xgbrcv.fit(X_train, y_train)
print("best params:", xgbrcv.best_params_)
gbpredY = xgbrcv.predict(X_test)
acc = metrics.accuracy_score(y_test, gbpredY)
print("gradient boosting test accuracy =", acc)
predictY_xgb = xgbrcv.predict(testtf)
# write_csv_kaggle_sub("my_submission4_xgb.csv", predictY_xgb)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 24.4min finished


best params: {'colsample_bytree': 0.8717140047046047, 'gamma': 0.42769285628251436, 'learning_rate': 0.05461018994592304, 'max_depth': 2, 'n_estimators': 926, 'subsample': 0.8285177730578381}
gradient boosting test accuracy = 0.7145833333333333


- The test accuracy is not well compared to svm classifier.
- The final result is much worse than I imagined, even worse than multinormial naive bayes classifier.

### Textblob
- **I found it when I search the relevant information. It seem insteresting.**
- TextBlob is an open source text processing library written in Python. It can be used to perform many natural language processing tasks, such as part-of-speech tagging, nominal component extraction, sentiment analysis, text translation, etc.
- It is easy to use to do sentiment analysis. If analysis.sentiment.polarity > 0, it means the input data represent positive, if analysis.sentiment.polarity = 0, it means neutral, and if analysis.sentiment.polarity < 0, it means negative.

In [38]:
from textblob import TextBlob
import re
Y = []
for i in range(0,len(testtxt)):
    analysis = TextBlob(testtxt[i])
    if analysis.sentiment.polarity > 0:
        Y.append("positive") 
    elif analysis.sentiment.polarity == 0:
        Y.append("neutral")
    else:
        Y.append("negative")
# write_csv_kaggle_sub("my_submission6.csv", Y)
print(Y)

['neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'positive', 'negative', 'negative', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'neutral', 'negative', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'ne

- But the final result is still not good as svm classifier.

### Use SVD to do feature extraction
- I also trie use SVD to do the feature extraction, but the result is too bad, it seems cannot predict the positive tweet, why? 

In [48]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(features)
xtrain_svd = svd.transform(features)
xtest_svd = svd.transform(testtf)
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xtest_svd_scl = scl.transform(xtest_svd)

In [49]:
clf = Pipeline([('vect', decomposition.TruncatedSVD()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', SVC(C=0.99, kernel = 'linear'))])
testclf = clf.fit(xtrain_svd_scl, trainY)
pred = testclf.predict(xtest_svd_scl)
print(pred)

['neutral' 'neutral' 'neutral' ... 'neutral' 'neutral' 'neutral']
